In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" -q
!pip install yfinance pandas numpy matplotlib seaborn scikit-learn -q
!pip install --upgrade transformers datasets accelerate bitsandbytes -q

# Tech Analysis library
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz -q
!tar -xzf ta-lib-0.4.0-src.tar.gz
!cd ta-lib && ./configure --prefix=/usr && make && make install
!pip install TA-Lib -q

print("✅ Installation complete! Please RESTART RUNTIME now!")
print("After restart, run the next cell...")

In [ ]:
import os
import pandas as pd
import numpy as np
import yfinance as yf
import json
from datetime import datetime, timedelta
from typing import List, Dict, Any
import warnings
warnings.filterwarnings('ignore')

# unsloth imports
from unsloth import FastLanguageModel
import torch
from datasets import Dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# Custom TA Funx
class TechnicalIndicators:
    @staticmethod
    def rsi(prices, period=14):
        """Calculate RSI"""
        delta = prices.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        return 100 - (100 / (1 + rs))

    @staticmethod
    def macd(prices, fast=12, slow=26, signal=9):
        """Calculate MACD"""
        ema_fast = prices.ewm(span=fast).mean()
        ema_slow = prices.ewm(span=slow).mean()
        macd_line = ema_fast - ema_slow
        signal_line = macd_line.ewm(span=signal).mean()
        return macd_line, signal_line

    @staticmethod
    def sma(prices, period):
        """Simple Moving Average"""
        return prices.rolling(window=period).mean()

print("🚀 Imports successful! Starting trading model setup...")

class StockDataProcessor:
    """Processes stock data and creates training dataset"""

    def __init__(self):
        # High-risk, high-reward stock tickers
        self.high_risk_tickers = [
            'TSLA', 'NVDA', 'AMD', 'PLTR', 'MSTR', 'COIN',
            'MRNA', 'BNTX', 'GME', 'AMC', 'SPCE', 'MARA',
            'RIOT', 'BABA', 'NIO'  # Reduced list for faster processing
        ]

    def fetch_stock_data(self, ticker: str, period: str = '1y') -> pd.DataFrame:
        """Fetch comprehensive stock data"""
        try:
            print(f"  📈 Fetching {ticker}...")
            stock = yf.Ticker(ticker)
            df = stock.history(period=period)

            if df.empty:
                return None

            # Custom technical indicators (no TA-Lib needed!)
            ta = TechnicalIndicators()

            df['RSI'] = ta.rsi(df['Close'])
            df['MACD'], df['MACD_signal'] = ta.macd(df['Close'])
            df['SMA_20'] = ta.sma(df['Close'], 20)
            df['SMA_50'] = ta.sma(df['Close'], 50)

            # Price changes and volatility
            df['Price_Change'] = df['Close'].pct_change()
            df['Volatility'] = df['Price_Change'].rolling(window=20).std()
            df['Volume_MA'] = df['Volume'].rolling(window=20).mean()
            df['Volume_Ratio'] = df['Volume'] / df['Volume_MA']

            # Future returns (target variable)
            df['Future_1d'] = df['Close'].shift(-1) / df['Close'] - 1
            df['Future_5d'] = df['Close'].shift(-5) / df['Close'] - 1

            return df

        except Exception as e:
            print(f"  ❌ Error fetching {ticker}: {e}")
            return None

    def create_training_prompt(self, row: pd.Series, ticker: str) -> Dict[str, str]:
        """Create training prompt from stock data"""

        current_price = row['Close']
        rsi = row['RSI']
        macd = row['MACD']
        volume_ratio = row['Volume_Ratio']
        volatility = row['Volatility'] * 100

        # Technical signals
        signals = []
        if rsi < 30:
            signals.append("oversold")
        elif rsi > 70:
            signals.append("overbought")

        if macd > row['MACD_signal']:
            signals.append("bullish MACD")
        else:
            signals.append("bearish MACD")

        # Determine action based on future returns
        future_1d = row['Future_1d']

        if pd.isna(future_1d):
            return None

        if future_1d > 0.03:
            action = "STRONG BUY"
        elif future_1d > 0.01:
            action = "BUY"
        elif future_1d < -0.03:
            action = "STRONG SELL"
        elif future_1d < -0.01:
            action = "SELL"
        else:
            action = "HOLD"

        prompt = f"""Analyze this high-risk stock:
Stock: {ticker}
Price: ${current_price:.2f}
RSI: {rsi:.1f}
MACD: {macd:.4f}
Volume: {volume_ratio:.2f}x normal
Volatility: {volatility:.2f}%
Signals: {', '.join(signals)}

Provide trading recommendation:"""

        response = f"""RECOMMENDATION: {action}
REASONING: {ticker} shows {', '.join(signals)} with {volatility:.2f}% volatility.
TARGET: {future_1d*100:+.2f}% expected 1-day move.
RISK: {'HIGH' if volatility > 3 else 'MEDIUM'} - use small position size."""

        return {
            "instruction": prompt,
            "output": response
        }

def prepare_training_data(max_samples: int = 1000) -> Dataset:
    """Prepare training dataset"""
    print("📊 Preparing training data...")
    processor = StockDataProcessor()
    training_data = []

    for ticker in processor.high_risk_tickers:
        df = processor.fetch_stock_data(ticker)

        if df is None or df.empty:
            continue

        # Sample data points
        sample_size = min(50, len(df.dropna()))  # 50 samples per stock
        sampled_df = df.dropna().tail(sample_size)  # Use recent data

        for idx, row in sampled_df.iterrows():
            prompt_data = processor.create_training_prompt(row, ticker)
            if prompt_data:
                formatted_prompt = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{prompt_data['instruction']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{prompt_data['output']}<|eot_id|>"""

                training_data.append({"text": formatted_prompt})

    print(f"✅ Created {len(training_data)} training samples")
    return Dataset.from_list(training_data)

def setup_and_train_model():
    """Setup and train the model"""
    print("🤖 Loading model...")

    # Load model
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "unsloth/llama-3.1-8b-instruct-bnb-4bit",
        max_seq_length = 2048,
        dtype = None,
        load_in_4bit = True,
    )

    # Setup LoRA
    model = FastLanguageModel.get_peft_model(
        model,
        r = 16,
        target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                          "gate_proj", "up_proj", "down_proj",],
        lora_alpha = 16,
        lora_dropout = 0,
        bias = "none",
        use_gradient_checkpointing = "unsloth",
        random_state = 3407,
    )

    print("📚 Preparing training data...")
    dataset = prepare_training_data()

    print("🏋️ Starting training...")
    trainer = SFTTrainer(
        model = model,
        tokenizer = tokenizer,
        train_dataset = dataset,
        dataset_text_field = "text",
        max_seq_length = 2048,
        dataset_num_proc = 1,
        packing = False,
        args = TrainingArguments(
            per_device_train_batch_size = 1,  # Small batch for Colab
            gradient_accumulation_steps = 8,
            warmup_steps = 10,
            max_steps = 50,  # Quick training
            learning_rate = 2e-4,
            fp16 = not is_bfloat16_supported(),
            bf16 = is_bfloat16_supported(),
            logging_steps = 5,
            optim = "adamw_8bit",
            weight_decay = 0.01,
            lr_scheduler_type = "linear",
            seed = 3407,
            output_dir = "outputs",
            dataloader_pin_memory = False,
        ),
    )

    trainer.train()

    print("💾 Saving model...")
    model.save_pretrained("trading_model")
    tokenizer.save_pretrained("trading_model")

    return model, tokenizer

def test_model(model, tokenizer):
    """Test the trained model"""
    print("\n🧪 Testing model...")

    test_prompt = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Analyze this high-risk stock:
Stock: TSLA
Price: $242.50
RSI: 45.2
MACD: 1.23
Volume: 1.8x normal
Volatility: 4.2%
Signals: neutral RSI, bullish MACD

Provide trading recommendation:<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""

    inputs = tokenizer(test_prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens = 256,
            temperature = 0.7,
            do_sample = True,
            pad_token_id = tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("📊 Sample Analysis:")
    print(response.split("<|start_header_id|>assistant<|end_header_id|>")[-1])

# Main execution
print("🎯 Starting High-Risk Trading Model Training")
print("=" * 50)

try:
    model, tokenizer = setup_and_train_model()
    test_model(model, tokenizer)
    print("\n✅ Training Complete!")
    print("🚀 Model ready for trading analysis!")

except Exception as e:
    print(f"❌ Error: {e}")
    print("💡 Make sure you restarted runtime after installation!")

In [ ]:
def analyze_live_stock(ticker: str):
    """Analyze a stock in real-time using custom technical indicators"""
    try:
        from unsloth import FastLanguageModel
        import yfinance as yf
        import torch

        # Load the trained model
        model, tokenizer = FastLanguageModel.from_pretrained(
            "trading_model",
            max_seq_length=2048,
            dtype=None,
            load_in_4bit=True,
        )

        # Get fresh data
        stock = yf.Ticker(ticker)
        df = stock.history(period='30d')
        if df.empty:
            print(f"❌ No data found for {ticker}")
            return

        latest = df.iloc[-1]

        # Calculate indicators using your class
        ta = TechnicalIndicators()
        rsi = ta.rsi(df['Close']).iloc[-1]
        macd_line, macd_signal = ta.macd(df['Close'])
        macd = macd_line.iloc[-1]
        macd_sig = macd_signal.iloc[-1]

        prompt = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Analyze this high-risk stock:
Stock: {ticker}
Price: ${latest['Close']:.2f}
RSI: {rsi:.1f}
MACD: {macd:.2f}
Volume: {latest['Volume']:.0f}

Provide trading recommendation:<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""

        device = "cuda" if torch.cuda.is_available() else "cpu"
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        model = model.to(device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=200,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id,
            )

        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        analysis = response.split("<|start_header_id|>assistant<|end_header_id|>")[-1]

        print(f"\n📊 Live Analysis for {ticker}:")
        print("=" * 30)
        print(analysis)

    except Exception as e:
        print(f"❌ Error analyzing {ticker}: {e}")

# RUN IT!!!
analyze_live_stock("")


In [ ]:
analyze_live_stock("BSGM")
analyze_live_stock("SCPH")
analyze_live_stock("")